In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [ ]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import csv
import math
import os
import pickle
import random
import re
import statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import bottleneck as bn
import numpy as np
import pandas as pd

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras.layers import (
    Activation,
    Add,
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling1D,
    Input,
    LayerNormalization,
    LSTM,
    Masking,
    MaxPooling1D,
    MultiHeadAttention,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


# プロットと可視化
import japanize_matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# その他のライブラリ
from fastdtw import fastdtw
from scipy import signal, stats
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from tslearn.metrics import dtw_path

In [ ]:
#motion_data = mr.process_apple_watch_csv('/Users/hinase/CodeChord/MasterResearch/datasets/new/nakazawa/motion/raw/nakazawa_eat.csv')
eye_data = mr.process_tobii_csv('datasets/new/mizuno/eye/raw/mizuno1_eat.csv')

In [ ]:
# 25秒減算坂本
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(25.659, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(25.4841, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(22.831, unit='s')
# 29秒減算渡部
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(29.644, unit='s')
# 43秒減算小寺
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(43.0029, unit='s')
# 43秒減算小寺(tri, check)
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(42.045, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(42.958, unit='s')
# 43秒減算中沢（circle, tri）
eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(38.43, unit='s')
# 43秒減算中沢（cross）
#eye_data["Timestamp"] = eye_data["Timestamp"] + pd.to_timedelta(11.486, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(43.66, unit='s')
# 43秒減算水野（cross）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(2.043, unit='s')
# 43秒減算水野（tri）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(2.171, unit='s')
# 43秒減算水野（circle, check）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(1.994, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(0.1, unit='s')

In [ ]:
type(eye_data["Timestamp"])

In [ ]:
motion_data["Timestamp"]

In [ ]:
Timestamp('2024-10-10 16:00:14.060896873')

In [ ]:
#hinase_circle_seg = mr.three_axis_spring(motion_data, train_data, [5, 5, 5], 'acc')

In [ ]:
Hz = 100

In [ ]:
a = mr.find_true_intervals(motion_data)

In [ ]:
a

In [ ]:
motion_data["Timestamp"][30895]

In [ ]:
#b = []

In [ ]:
print(i)

In [ ]:
i = i + 1

In [ ]:
i

In [ ]:
a[i]

In [ ]:
b.append([a[i][0]+80, a[i][1]-130])

In [ ]:
b

In [ ]:
len(b)

In [ ]:
del b[11]

In [ ]:
for j in range(len(b)):
  print(b[j][1] - b[j][0])

In [ ]:
for i in range(len(a)):
  print(a[i][1] - a[i][0])

In [ ]:
len(b)

In [ ]:
del b[10]

In [ ]:
c = []
for i in range(len(b)):
  c.append([motion_data["Timestamp"][b[i][0]], motion_data["Timestamp"][b[i][1]]])

In [ ]:
c

In [ ]:
# 各範囲内のデータを格納するリストを用意します
result = []
# cの各行に対してループを回します
for start_time, end_time in extracted_data:
    # 範囲内のデータを抽出するためのマスクを作成します
    mask = (eye_data["Timestamp"] >= start_time) & (eye_data["Timestamp"] <= end_time)
    # マスクを適用してデータを抽出します
    data_in_range = eye_data.loc[mask]
    # 結果をリストに追加します
    result.append(data_in_range)

# 結果を確認します
for idx, data in enumerate(result):
    print(f"範囲 {idx+1}:")
    print(data)
    print("\n")

In [ ]:
# 定数の定義

LABELS = ['circle', 'cross', 'tri', 'check']
COLUMNS_TO_DROP = [
    'Sensor', 'Participant name', 'Event', 'Event value',
    'Eye movement type', 'Eye movement type index', 'Ungrouped',
    'Validity left', 'Validity right', 'Gaze event duration', 'Gaze2D_Distance',
    'Fixation_Distance', 'Gaze3D_Distance', 'Pupil_Diameter_Change',
    'GazeDirection_Distance', 'PupilPosition_Distance'
]
# インターバルのパラメータ設定
MIN_INTERVAL = 0.84  # 最小インターバル長（秒）
MAX_INTERVAL = 3.04  # 最大インターバル長（秒）
NUM_INTERVALS = 160  # 抽出するインターバルの数

In [ ]:
for name in NAMES:
    print(f"Processing {name}...")

    # データの読み込みと前処理
    eye_data, gesture_labels = mr.load_gesture_eye_data_pickle(name, LABELS, COLUMNS_TO_DROP)
    # 変数に保存
    globals()[f'{name}_eye'] = eye_data
    globals()[f'{name}_label'] = gesture_labels
    globals()[f'{name}_true_label'] = ['gesture'] * len(eye_data)

    # モーションデータとアイデータの読み込みと前処理
    motion_routine_data, eye_routine_data = mr.load_motion_and_eye_data(name)
    # 変数に保存
    globals()[f'{name}_motion_routine'] = motion_routine_data
    globals()[f'{name}_eye_routine'] = eye_routine_data

    # 'Timestamp' カラムの存在と型を確認
    assert 'Timestamp' in motion_routine_data.columns, f"{name}_motion_routineに'Timestamp'がありません。"
    assert motion_routine_data['Timestamp'].dtype == 'datetime64[ns]', f"{name}_motion_routineの'Timestamp'がdatetime64[ns]ではありません。"
    assert 'Timestamp' in eye_routine_data.columns, f"{name}_eye_routineに'Timestamp'がありません。"
    assert eye_routine_data['Timestamp'].dtype == 'datetime64[ns]', f"{name}_eye_routineの'Timestamp'がdatetime64[ns]ではありません。"

    # 既に読み込んだモーションデータを使用
    motion_data = motion_routine_data

    # 除外インターバルの取得
    exclusion_intervals = mr.get_exclusion_intervals(motion_data)

    # データ全体の時間範囲の取得
    start_time = motion_routine_data['Timestamp'].min()
    end_time = motion_routine_data['Timestamp'].max()

    # 利用可能なインターバルの取得
    available_intervals = mr.get_available_intervals(exclusion_intervals, start_time, end_time)

    print(f"{name} の利用可能なインターバル数: {len(available_intervals)}")

    # ランダムなインターバルの抽出（eye_routine_data を使用）
    extracted_data = mr.extract_random_intervals(
        eye_routine_data, available_intervals, MIN_INTERVAL, MAX_INTERVAL, NUM_INTERVALS
    )

    # 抽出されたデータを変数に保存
    globals()[f'{name}_extracted_intervals'] = extracted_data
    globals()[f'{name}_false_label'] = ['non_gesture'] * len(extracted_data)

In [ ]:
print(mizuno_motion_routine['Timestamp'].dtype)

In [ ]:
print(mizuno_eye_routine['Timestamp'].dtype)

In [ ]:
# データとラベルを格納するリストの初期化
X = []
y_labels = []
true_labels = []

for name in NAMES:
    # ジェスチャーの目のデータと対応するラベルを取得
    eye_data = globals()[f'{name}_eye']  # リスト形式のデータフレーム
    gesture_labels = globals()[f'{name}_label']  # 個別のジェスチャーラベル
    true_label_gesture = globals()[f'{name}_true_label']  # ['gesture'] * len(eye_data)

    # シーケンスが空でないか確認しながらデータを追加
    for seq, lbl, true_lbl in zip(eye_data, gesture_labels, true_label_gesture):
        if seq is not None and len(seq) > 0:
            X.append(seq)
            y_labels.append(lbl)
            true_labels.append(true_lbl)
        else:
            print(f"{name} のジェスチャーシーケンスが空です。対応するラベルをスキップします。")

    # 非ジェスチャーの目のデータと対応するラベルを取得
    extracted_data = globals()[f'{name}_extracted_intervals']  # リスト形式のデータフレーム
    false_labels = globals()[f'{name}_false_label']  # ['non_gesture'] * len(extracted_data)
    true_label_non_gesture = globals()[f'{name}_false_label']  # ['non_gesture'] * len(extracted_data)

    # シーケンスが空でないか確認しながらデータを追加
    for seq, lbl, true_lbl in zip(extracted_data, false_labels, true_label_non_gesture):
        if seq is not None and len(seq) > 0:
            X.append(seq)
            y_labels.append(lbl)
            true_labels.append(true_lbl)
        else:
            print(f"{name} の非ジェスチャーシーケンスが空です。対応するラベルをスキップします。")

# データの長さを確認
print(f"総データ数: {len(X)}")
print(f"総ラベル数（y_labels）: {len(y_labels)}")
print(f"総真偽ラベル数（true_labels）: {len(true_labels)}")

In [ ]:
length = []
for i in range(len(X)):
  length.append(len(X[i]))

In [ ]:
len(X[])

In [ ]:
X[0].columns

In [ ]:
X =[]
X.extend(sakamoto_eye)
X.extend(watabe_eye)
X.extend(nakazawa_eye)
X.extend(kotera_eye)
X.extend(mizuno_eye)

In [ ]:
label = []
label.extend(sakamoto_label)
label.extend(watabe_label)
label.extend(nakazawa_label)
label.extend(kotera_label)
label.extend(mizuno_label)

In [ ]:
X_filled = []
X_scaled = []
for idx, sequence in enumerate(X):
    df = pd.DataFrame(sequence)
    # 欠損値の補間と補完
    df = df.interpolate(method='linear', limit_direction='both', axis=0)
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    # 残る NaN を 0 で埋める
    if df.isnull().values.any():
        print(f"シーケンス {idx} にまだ NaN が存在します。0 で埋めます。")
        df = df.fillna(0)
    # データ型を数値型に変換（必要に応じて）
    df = df.apply(pd.to_numeric, errors='coerce')
    X_filled.append(df.values)

# スケーリング前に NaN をチェック
for idx, sequence in enumerate(X_filled):
    if np.isnan(sequence).any():
        print(f"シーケンス {idx} に NaN が残っています。スケーリングをスキップします。")
        continue
    if sequence.shape[0] > 0:
        scaler = StandardScaler()
        X_scaled.append(scaler.fit_transform(sequence))
    else:
        print(f"シーケンス {idx} は空です。スケーリングをスキップします。")


In [ ]:
len(X_scaled)

In [ ]:
# 1. ラベルのエンコーディング
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(label)
num_classes = len(label_encoder.classes_)
y_categorical = to_categorical(y_encoded, num_classes=num_classes)

# 2. データの分割（パディングの前に行う）
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.3, random_state=42
)

# 3. 訓練データとテストデータでそれぞれ最大シーケンス長を計算
max_length_train = max(len(seq) for seq in X_train_raw)
max_length_test = max(len(seq) for seq in X_test_raw)
max_length = max(max_length_train, max_length_test)

# 4. シーケンスのパディング
X_train_padded = pad_sequences(X_train_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')
X_test_padded = pad_sequences(X_test_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')

# 5. パディング後のデータに NaN が含まれていないか確認
print('NaN in X_train_padded:', np.isnan(X_train_padded).any())
print('NaN in X_test_padded:', np.isnan(X_test_padded).any())

# 必要に応じて NaN を 0 で置換
if np.isnan(X_train_padded).any():
    X_train_padded = np.nan_to_num(X_train_padded, nan=0.0)
if np.isnan(X_test_padded).any():
    X_test_padded = np.nan_to_num(X_test_padded, nan=0.0)

In [ ]:
print(f'Number of samples in X_scaled: {len(X_scaled)}')
print(f'Number of samples in y_categorical: {len(y_categorical)}')


In [ ]:
# 特徴量数を取得
num_features = X_train_padded.shape[2]
num_features

In [ ]:
# 4. モデルの構築
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_length, num_features)))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='sigmoid'))

# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
input_shape = (max_length, num_features)
inputs = Input(shape=input_shape)
x = Masking(mask_value=0.)(inputs)

# num_featuresを定義（入力データの特徴量の次元数）
num_features = input_shape[1]

# Transformerブロックの定義
def transformer_block(x, num_heads, key_dim, ff_dim, rate=0.1):
    # マルチヘッド注意機構
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(rate)(attn_output)

    # 出力次元をnum_featuresに変換
    attn_output = Dense(num_features)(attn_output)

    out1 = Add()([x, attn_output])
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    # フィードフォワードネットワーク
    ffn_output = Dense(ff_dim, activation='relu')(out1)

    # 出力次元をnum_featuresに変換
    ffn_output = Dense(num_features)(ffn_output)

    ffn_output = Dropout(rate)(ffn_output)
    out2 = Add()([out1, ffn_output])
    out2 = LayerNormalization(epsilon=1e-6)(out2)
    return out2

# Transformerブロックの適用
x = transformer_block(x, num_heads=4, key_dim=64, ff_dim=128)

# プーリングと出力層
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes, activation='sigmoid')(x)

# モデルの作成
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)


In [ ]:
# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 8. 特徴量の重要度評価（Permutation Feature Importance）

# 8.1 ベースラインの性能を計算
# テストデータでの予測
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# ベースラインのAccuracyを計算
baseline_accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f'Baseline Accuracy: {baseline_accuracy:.4f}')

# 8.2 特徴量ごとの重要度を計算
num_features = X_test_padded.shape[2]
feature_importances = []

for feature_idx in range(num_features):
    # テストデータをコピー
    X_test_permuted = X_test_padded.copy()

    # 特徴量をランダムにシャッフル（サンプル間でシャッフル）
    permuted_feature = X_test_permuted[:, :, feature_idx].reshape(X_test_permuted.shape[0], -1)
    np.random.shuffle(permuted_feature)
    X_test_permuted[:, :, feature_idx] = permuted_feature.reshape(X_test_permuted.shape[0], X_test_permuted.shape[1])

    # シャッフル後のデータで予測
    y_pred_permuted = model.predict(X_test_permuted)
    y_pred_classes_permuted = np.argmax(y_pred_permuted, axis=1)

    # シャッフル後のAccuracyを計算
    permuted_accuracy = accuracy_score(y_true_classes, y_pred_classes_permuted)

    # 性能の差を計算（重要度）
    importance = baseline_accuracy - permuted_accuracy
    feature_importances.append(importance)
    print(f'Feature {feature_idx + 1} Importance: {importance:.4f}')

# 8.3 特徴量重要度の可視化
feature_names = [f'Feature {i+1}' for i in range(num_features)]

plt.figure(figsize=(10, 6))
plt.barh(X[7].columns, feature_importances)
plt.xlabel('Decrease in Accuracy')
plt.title('Permutation Feature Importance')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/d_nakazawa_acc_check_segments4.7new.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['d_nakazawa_check_segx']
segy = data['d_nakazawa_check_segy']
segz = data['d_nakazawa_check_segz']

In [ ]:
# Create a simple sine wave as an example of a waveform or signal
x = np.linspace(0, 10, 1000)
y = np.sin(x)

# Define the sections to be colored
color_sections = [(200, 300, 'red'), (450, 550, 'blue'), (700, 800, 'red')]

# Plot the entire waveform in gray
plt.figure(figsize=(10, 2))
plt.plot(x, y, color='gray', linewidth=3)

# Highlight the specified sections with the chosen colors
for start, end, color in color_sections:
    plt.plot(x[start:end], y[start:end], color=color, linewidth=5)

# Add an arrow to indicate the flow of data
# plt.annotate('', xy=(10, 0), xytext=(0, 0),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))
plt.xticks(color="None")
plt.yticks(color="None")
plt.tick_params(length=0)
# Remove axes for a cleaner look
plt.axis('off')
#plt.savefig("/Users/hinase/Downloads/plt.svg")

# Show plot
plt.show()

In [ ]:
import cairosvg

# SVGファイルのパス
input_svg_path = "/Users/hinase/Downloads/ss.svg"
# 出力するEPSファイルのパス
output_eps_path = "/Users/hinase/Downloads/ss.eps"

# SVGをEPSに変換
cairosvg.svg2eps(url=input_svg_path, write_to=output_eps_path)

print(f"SVG画像がEPS形式で '{output_eps_path}' に保存されました。")


In [ ]:
import os
import csv
import openpyxl
from natsort import natsorted

folder_path = '/Users/hinase/Downloads/folder3'
folder_2_path = '/Users/hinase/Downloads/folder4'

# Get Excel file names in natural order, filtering only supported Excel files
supported_extensions = ('.xlsx', '.xlsm', '.xltx', '.xltm')
files = natsorted([f for f in os.listdir(folder_path) if f.endswith(supported_extensions)])

# Loop over filenames and build absolute path for each file
for filename in files:
    filepath = os.path.join(folder_path, filename)

    # Access xlsx file and get the first sheet object
    wb = openpyxl.load_workbook(filepath)
    ws_name = wb.sheetnames[0]
    ws = wb[ws_name]

    # Convert to csv and save to folder4
    savecsv_path = os.path.join(folder_2_path, filename.rstrip(".xlsx") + ".csv")
    with open(savecsv_path, 'w', newline="") as csvfile:
        writer = csv.writer(csvfile)
        for row in ws.rows:
            writer.writerow([cell.value for cell in row])
